In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
class Model:
    
    def __init__(self,sess,name):
        self.sess = sess
        self.name = name
        self._build_net()
        
    def _build_net(self):
        with tf.variable_scope(self.name):
            # input place holder
            self.X = tf.placeholder(tf.float32, [None,784])
            
            self.keep_prob = tf.placeholder(tf.float32)
            self.training = tf.placeholder(tf.bool)
            
            # img 28 x 28 x 1(black/white)
            X_img = tf.reshape(self.X, [-1, 28, 28, 1])
            
            # 0~9
            self.Y = tf.placeholder(tf.float32, [None,10])
            
            # L1 Img In Shape = (?,28,28,1)
            conv1 = tf.layers.conv2d(inputs=X_img, filters=32, kernel_size=[3,3], padding='SAME', activation=tf.nn.relu)
            pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2,2], strides=2, padding='SAME')
            dropout1 = tf.layers.dropout(inputs=pool1, rate=self.keep_prob, training=self.training)

            # L2 Img In Shape = (?,14,14,1)
            conv2 = tf.layers.conv2d(inputs=dropout1, filters=64, kernel_size=[3,3], padding='SAME', activation=tf.nn.relu)
            pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2,2], strides=2, padding='SAME')
            dropout2 = tf.layers.dropout(inputs=pool2, rate=self.keep_prob, training=self.training)

            # L3 Img In Shape = (?,7,7,1)
            conv3 = tf.layers.conv2d(inputs=dropout2, filters=128, kernel_size=[3,3], padding='SAME', activation=tf.nn.relu)
            pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=[2,2], strides=2,padding='SAME')
            dropout3 = tf.layers.dropout(inputs=pool3, rate=self.keep_prob, training=self.training)

            layer = tf.reshape(dropout3, [-1,128*4*4])
            
            # 
            W4 = tf.get_variable("W4",shape=[128*4*4,625],initializer=tf.contrib.layers.xavier_initializer())
            b4 = tf.Variable(tf.random_normal([625]))
            L4 = tf.nn.relu(tf.matmul(layer,W4) + b4)
            L4 = tf.nn.dropout(L4, keep_prob=self.keep_prob)

            W5 = tf.get_variable("W5",shape=[625,10],initializer=tf.contrib.layers.xavier_initializer())
            b5 = tf.Variable(tf.random_normal([10]))
            self.hypothesis = tf.matmul(L4,W5)+b5

        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=self.hypothesis,labels=self.Y))
        self.optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(self.cost)

        prediction = tf.equal(tf.argmax(self.hypothesis,1),tf.argmax(self.Y,1))
        self.accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
            
            
    def prediction(self,x_test,keep_prob=1.0):
        return self.sess.run(self.hypothesis,feed_dict={self.X:x_test, self.keep_prob:keep_prob, self.training:False})

    def get_accuracy(self,x_test, y_test, keep_prob=1.0):
        return self.sess.run(self.accuracy,feed_dict={self.X:x_test,self.Y:y_test,self.keep_prob:keep_prob, self.training:False})

    def train(self, x_train, y_train, keep_prob=0.7):
        return self.sess.run([self.cost,self.optimizer], feed_dict={self.X:x_train, self.Y:y_train, self.keep_prob:keep_prob, self.training:True})


In [4]:
training_epochs = 15
batch_size = 100

In [5]:
tf.reset_default_graph()

sess = tf.Session()
m1 = Model(sess,"m1")

sess.run(tf.global_variables_initializer())

In [6]:
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples / batch_size)
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        c, _, = m1.train(batch_xs,batch_ys)
        avg_cost += c/total_batch
    print('Epoch : ', '%04d' % (epoch+1), 'cost =', '{:.9f}'.format(avg_cost))

Epoch :  0001 cost = 0.694089278
Epoch :  0002 cost = 0.270423071
Epoch :  0003 cost = 0.210346014
Epoch :  0004 cost = 0.184630169
Epoch :  0005 cost = 0.165917298
Epoch :  0006 cost = 0.159942421
Epoch :  0007 cost = 0.154416663
Epoch :  0008 cost = 0.142720303
Epoch :  0009 cost = 0.138381703
Epoch :  0010 cost = 0.131920195
Epoch :  0011 cost = 0.134341531
Epoch :  0012 cost = 0.131592467
Epoch :  0013 cost = 0.129377413
Epoch :  0014 cost = 0.125080048
Epoch :  0015 cost = 0.125910522


In [7]:
accuracy = m1.get_accuracy(mnist.test.images, mnist.test.labels)
print(accuracy)

0.987
